In [8]:
import polars as pl
from itertools import permutations, product
from tqdm import tqdm
import os

In [10]:
try:
    os.mkdir("_data")
except FileExistsError:
    pass

In [2]:
# Initialise it all
base_df = (
    pl.DataFrame(
        list(permutations(range(1, 11))), 
        orient="row", 
        schema=[f"slot_{i}" for i in range(10)]
    )
    .cast(pl.Int8)
)

df_dict = {}
for included in tqdm(product([False, True], repeat=10)):
    df_dict[included] = base_df.with_columns([
        pl.lit(-1, dtype=pl.Int8).alias(f"slot_{slot_num}")
        for slot_num in range(10) if not included[slot_num]
    ]).unique()

1024it [00:51, 19.85it/s]


In [3]:
# 1. Fill in the final states
# NOTE: numbering goes top to bottom, left to right, e.g., slots 0 to 2 are the first column. 
base_key = tuple([True] * 10)
df_dict[base_key] = df_dict[base_key].with_columns(
    ((
        (
            ((pl.col("slot_9") % 2) == (pl.col("slot_6") % 2)) 
            == (pl.col("slot_0") < pl.col("slot_3"))
        ).cast(pl.Int64)
        + (
            ((pl.col("slot_9") % 2) == (pl.col("slot_7") % 2)) 
            == (pl.col("slot_1") < pl.col("slot_4"))
        ).cast(pl.Int64)
        + (
            ((pl.col("slot_9") % 2) == (pl.col("slot_8") % 2)) 
            == (pl.col("slot_2") < pl.col("slot_5"))
        ).cast(pl.Int64)
    ) > 1.5).alias("player_1_wins")
)
(
    df_dict[base_key]
    .with_columns(
        pl.lit(None, dtype=pl.Int64).alias("target_square"),
        pl.lit(None, dtype=pl.Int8).alias("number_input")
    )
    .write_parquet("_data/tttttttttt.parquet")
)
del df_dict[base_key]

In [4]:
# 2. Fill in everything else by working backwards
for num_slots_filled in range(9, 0, -1):
    permutations = [l for l in product([False, True], repeat=10) if sum(l) == num_slots_filled]
    for slots_filled in tqdm(permutations):
        empty_slots = [i for i in range(10) if not slots_filled[i]]
        key_cols = [f"slot_{i}" for i in range(10) if slots_filled[i]]
        temp = (
            df_dict[slots_filled].lazy().join( # Making it all LazyFrames really speeds things up
                # For all possible moves in current state, get result
                pl.concat([
                    (
                        # Keep previous ones on disk rather than in RAM - avoids OOM issues
                        pl.scan_parquet(f"_data/{''.join(['t' if (slots_filled[i] or i == target_square) else 'f' for i in range(10)])}.parquet")
                        .with_columns(
                            pl.lit(target_square, dtype=pl.Int64).alias("target_square"),
                            pl.col(f"slot_{target_square}").alias("number_input")
                        )
                    )
                    for target_square in empty_slots
                ]),
                on=key_cols,
                how="left"
            )
            .sort("player_1_wins", descending=num_slots_filled % 2 == 0)
            .select(pl.selectors.exclude("^.*_right$"))
            .group_by(f"slot_{i}" for i in range(10))
            .first() # Keeps first row per group, which is the best after sorting
            .collect()
        )
        file_code = "".join(["t" if slots_filled[i] else "f" for i in range(10)])
        temp.write_parquet(f"_data/{file_code}.parquet")
        del df_dict[slots_filled]
    print(f"done for permutations {num_slots_filled}")

100%|███████████████████████████████████████████| 10/10 [00:05<00:00,  1.77it/s]


done for permutations 9


100%|███████████████████████████████████████████| 45/45 [00:37<00:00,  1.19it/s]


done for permutations 8


100%|█████████████████████████████████████████| 120/120 [01:07<00:00,  1.77it/s]


...

done for permutations 3


100%|███████████████████████████████████████████| 45/45 [00:01<00:00, 33.15it/s]


done for permutations 2


100%|███████████████████████████████████████████| 10/10 [00:00<00:00, 20.40it/s]

done for permutations 1


In [5]:
# 3. Special treatment for final case
empty_slots = list(range(10))
slots_filled = [False] * 10
dfs = [
    (
        pl.read_parquet(f"_data/{''.join(['t' if (slots_filled[i] or i == target_square) else 'f' for i in range(10)])}.parquet")
        .with_columns(
            pl.lit(target_square, dtype=pl.Int64).alias("target_square"),
            pl.col(f"slot_{target_square}").alias("number_input")
        )
    )
    for target_square in empty_slots
]
cols = dfs[0].columns
initial_pos = pl.concat([d.select(cols) for d in dfs]).sort("player_1_wins", descending=True)[:1]
for i in range(10):
    initial_pos[0, i] = -1
initial_pos.write_parquet("_data/ffffffffff.parquet")